# Communication policy

Load data

In [3]:
import json

def load_pairs(data_path):
    with open(data_path, "r") as f:
        data = json.load(f)
    pairs = []
    for step, s_data in data.items():
        if not step.startswith("Step"):
            continue
        pairs.append({
            "observation": s_data["Agent_0"]["Observation"],
            "sentence": s_data["Agent_0"]["Sentence"][1:-1]
        })
        pairs.append({
            "observation": s_data["Agent_1"]["Observation"],
            "sentence": s_data["Agent_1"]["Sentence"][1:-1]
        })
    return pairs

data_pairs = load_pairs("test_data/Sentences_Generated_P1.json")

train_data = data_pairs[:80000]
test_data = data_pairs[80000:]

In [5]:
import torch

from model.modules.obs import ObservationEncoder
from model.modules.lm import GRUEncoder, OneHotEncoder
from model.modules.comm import CommunicationPolicy

cp = CommunicationPolicy(16, 32)

word_encoder = OneHotEncoder(['South','Not','Located','West','Object','Landmark','North','Center','East'])
lang_enc = GRUEncoder(32, word_encoder)
obs_enc = ObservationEncoder(17, 32)
dec = GRUDecoder(32, word_encoder)

obs = torch.Tensor([train_data[0]["observation"]])
sent = [train_data[0]["sentence"]]

int_context = obs_enc(obs)
ext_context = lang_enc(sent)
print(int_context)
print(ext_context)

NameError: name 'GRUDecoder' is not defined

In [34]:
int_context = int_context / int_context.norm(dim=1, keepdim=True)
ext_context = ext_context.squeeze(0)
ext_context = ext_context / ext_context.norm(dim=1, keepdim=True)
sim = int_context @ ext_context.t()
sim

tensor([[0.9645]], grad_fn=<MmBackward0>)

In [20]:
import torch
a = torch.ones((1, 20))
a

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.]])

In [8]:
torch.conca((a, a)).shape

torch.Size([2, 1, 20])

In [12]:
torch.cat([a,a]).shape

torch.Size([2, 20])